In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers datasets scikit-learn torch

In [ ]:
!pip install GPUtil
# most likely u dont need this
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           


# Models

In [ ]:
import torch
from transformers import BertModel, BertPreTrainedModel
from torch import nn

class BERTCNN(BertPreTrainedModel):
    def __init__(self, config):
        super(BERTCNN, self).__init__(config)
        self.bert = BertModel.from_pretrained("bert-base-uncased", config=config)
        self.conv = nn.Conv1d(in_channels=config.hidden_size, out_channels=128, kernel_size=5, padding=2)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(128, config.num_labels)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        sequence_output = outputs[0]
        sequence_output = sequence_output.permute(0, 2, 1)
        x = self.conv(sequence_output)
        x = self.pool(x).squeeze(-1)
        x = self.dropout(x)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits

In [ ]:
import torch
from torch import nn
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer, TrainingArguments, Trainer

class TransBLSTM(BertPreTrainedModel):
    def __init__(self, config):
        super(TransBLSTM, self).__init__(config)
        self.bert = BertModel.from_pretrained("bert-base-uncased", config=config)
        self.blstm = nn.LSTM(config.hidden_size, config.hidden_size // 2, 
                             num_layers=1, bidirectional=True, batch_first=True)
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.dropout = nn.Dropout(0.5)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, 
                position_ids=None, head_mask=None, inputs_embeds=None, labels=None):
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask, 
                                 token_type_ids=token_type_ids, position_ids=position_ids, 
                                 head_mask=head_mask, inputs_embeds=inputs_embeds)
        
        sequence_output = bert_outputs[0]
        blstm_output, _ = self.blstm(sequence_output)
        combined_output = self.layer_norm(sequence_output + blstm_output)
        
        pooled_output = combined_output[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))
        return (loss, logits) if loss is not None else logits


In [ ]:
from transformers import AutoModelForSequenceClassification

class RoBERTa:
    def __init__(self, model_type='cardiffnlp/twitter-roberta-base-sentiment', num_labels=3):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_type, num_labels=num_labels)

    def get_model(self):
        return self.model

In [ ]:
!pip install transformers[torch] accelerate -U

# Training & Data Cleanup

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from datasets import Dataset
import torch

def load_and_balance_data(file_path, target_count=10000):
    df = pd.read_csv(file_path, on_bad_lines='skip', nrows=100000)
    df['Sentiment'] = df['Score'].apply(map_score_to_sentiment)
    
    # Ensure samples
    def ensure_samples(data, target_count):
        if len(data) >= target_count:
            return data.sample(target_count, random_state=42)
        else:
            return data.sample(target_count, replace=True, random_state=42)
    
    negative_df = ensure_samples(df[df['Sentiment'] == 0], target_count)
    neutral_df = ensure_samples(df[df['Sentiment'] == 1], target_count)
    positive_df = ensure_samples(df[df['Sentiment'] == 2], target_count)
    
    balanced_df = pd.concat([negative_df, neutral_df, positive_df]).sample(frac=1, random_state=42)  # Shuffle dataset
    
    train_df, temp_df = train_test_split(balanced_df[['Text', 'Sentiment']], test_size=0.3, random_state=42)
    test_df, val_df = train_test_split(temp_df, test_size=1/3, random_state=42)
    
    return train_df, test_df, val_df

def map_score_to_sentiment(score):
    return 0 if score < 3 else (1 if score == 3 else 2)

def save_to_csv(data_df, file_name):
    data_df.to_csv(file_name, index=False)

def tokenize_data(tokenizer, texts, labels):
    tokenized_inputs = tokenizer(texts.tolist(), padding="max_length", truncation=True, max_length=512)
    return Dataset.from_dict({**tokenized_inputs, 'labels': labels.tolist()})

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"f1": f1_score(p.label_ids, preds, average='macro')}

def train_model(model, train_dataset, val_dataset, tokenizer, output_dir):
    training_args = TrainingArguments(
        output_dir=output_dir,
        report_to="none",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        save_total_limit=2,
        save_steps=500,
        eval_strategy="steps",
        eval_steps=500,
        load_best_model_at_end=True,
        metric_for_best_model="loss" 
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

    model_path = f"{output_dir}/best_model"
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

In [ ]:
train_df, test_df, val_df = load_and_balance_data('/kaggle/input/amazon-fine-food-reviews/Reviews.csv')

save_to_csv(train_df, '/kaggle/working/balanced_train_data.csv')
save_to_csv(test_df, '/kaggle/working/balanced_test_data.csv')
save_to_csv(val_df, '/kaggle/working/balanced_val_data.csv')

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_dataset = tokenize_data(tokenizer, train_df['Text'], train_df['Sentiment'])
test_dataset = tokenize_data(tokenizer, test_df['Text'], test_df['Sentiment'])
val_dataset = tokenize_data(tokenizer, val_df['Text'], val_df['Sentiment'])


def data_stats(data_df):
    num_reviews = data_df.shape[0]
    print(f"Total number of reviews: {num_reviews}")

    lengths = data_df['Text'].apply(len)
    average_length = lengths.mean()
    print(f"Average review length: {average_length:.2f} characters")

    all_words = ' '.join(data_df['Text']).split()
    vocab_size = len(set(all_words))
    print(f"Vocabulary size: {vocab_size}")

    median_length = lengths.median()
    min_length = lengths.min()
    max_length = lengths.max()
    print(f"Median review length: {median_length} characters")
    print(f"Minimum review length: {min_length} characters")
    print(f"Maximum review length: {max_length} characters")

    negative_count = (data_df['Sentiment'] == 0).sum()
    neutral_count = (data_df['Sentiment'] == 1).sum()
    positive_count = (data_df['Sentiment'] == 2).sum()
    print(f'Negative Count: {negative_count}')
    print(f'Neutral Count: {neutral_count}')
    print(f'Positive Count: {positive_count}')

print("Training Data Statistics:")
data_stats(train_df)
print("\nTesting Data Statistics:")
data_stats(test_df)
print("Validation Data Statistics:")
data_stats(val_df)


# Train

In [ ]:
bert_model_type = 'bert-base-uncased'
bert_cnn_config = BertConfig.from_pretrained(bert_model_type, num_labels=3)
bert_cnn_model = BERTCNN(config=bert_cnn_config)
train_model(bert_cnn_model, train_dataset, val_dataset, tokenizer, './bert_cnn_results')

In [ ]:
bert_model_type = "bert-base-uncased"
trans_blstm_config = BertConfig.from_pretrained(bert_model_type, num_labels=3)
trans_blstm_model =  TransBLSTM.from_pretrained(bert_model_type, config=trans_blstm_config)
train_model(trans_blstm_model, train_dataset, val_dataset, tokenizer, "./trans_blstm_model")

In [ ]:
roberta_model_type = 'cardiffnlp/twitter-roberta-base-sentiment'
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_type)
roberta_train_dataset = tokenize_data(roberta_tokenizer,  train_df['Text'], train_df['Sentiment'])
roberta_test_dataset = tokenize_data(roberta_tokenizer,  test_df['Text'], test_df['Sentiment'])
roberta_val_dataset = tokenize_data(roberta_tokenizer, val_df['Text'], val_df['Sentiment'])

roberta_model = RoBERTa(model_type=roberta_model_type).get_model()
train_model(roberta_model, roberta_train_dataset, roberta_test_dataset, roberta_val_dataset, './roberta_results')

# Optionally Load Models

In [ ]:
# model_dir = "/kaggle/working/bert_cnn_results/best_model/"

# loaded_bert_cnn_config = BertConfig.from_pretrained(model_dir)
# loaded_bert_cnn_model = BERTCNN.from_pretrained(model_dir, config=config)

# Download

In [ ]:
# !zip -r roberta_results.zip /kaggle/working/roberta_results/best_model/

In [ ]:
# !cd roberta_results

In [ ]:
# !ls

In [ ]:
# from IPython.display import FileLink
# FileLink(r'roberta_results.zip')

# Test

In [ ]:
import numpy as np

In [ ]:
def test_model(model, test_dataset):
    trainer = Trainer(model=model)
    result = trainer.predict(test_dataset)
    prediction = np.argmax(result.predictions, axis=1)
    return result, prediction

In [ ]:
# bert_cnn_result, bert_cnn_preds = test_model(bert_cnn_model, test_dataset) # from runtime
bert_cnn_result, bert_cnn_preds = test_model(loaded_bert_cnn_model, test_dataset) # from directory

In [ ]:
trans_blstm_result, trans_blstm_preds = test_model(trans_blstm_model, test_dataset)

In [ ]:
roberta_result, roberta_preds = test_model(roberta_model, roberta_test_dataset)

# Evaluate

In [ ]:
def calculate_f1(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    if (precision + recall) == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)


In [ ]:
def compare_deprecated(model_result, model_preds, model_type):
    print(model_type)
    cases = ['negative', 'neutral', 'positive']

    predictions_map = {
        'negative': [],
        'neutral': [],
        'positive': [],
    }
    truth_map = {
        'negative': [],
        'neutral': [],
        'positive': [],
    }
    falsy_map = {
        'negative': [],
        'neutral': [],
        'positive': [],
    }

    for i, (result, preds) in enumerate(zip(model_result, model_preds)):
        score = test_df['Sentiment'].iloc[i]
        truth_map[cases[score]].append(i)
        predictions_map[cases[preds]].append(i)
        if score > 0 and preds == 0:
            falsy_map[cases[0]].append(i)
        elif score != 1 and preds == 1:
            falsy_map[cases[1]].append(i)
        elif score <2 and preds == 2:
            falsy_map[cases[2]].append(i)


    total_data = len(predictions_map[cases[0]]) + len(predictions_map[cases[1]]) + len(predictions_map[cases[2]])

    print("Predictions")
    print(f'Negative:{len(predictions_map[cases[0]])} | Neutral: {len(predictions_map[cases[1]])} | Positive: {len(predictions_map[cases[2]])}')
    print("============\n")
    print("Truth")
    print(f'Negative:{len(truth_map[cases[0]])} | Neutral: {len(truth_map[cases[1]])} | Positive: {len(truth_map[cases[2]])}')
    print("============\n")
    print("False Positives")
    print(f'Negative:{len(falsy_map[cases[0]])} ({len(falsy_map[cases[0]])/len(truth_map[cases[0]])*100})| Neutral: {len(falsy_map[cases[1]])} ({len(falsy_map[cases[1]])/len(truth_map[cases[1]])*100})| Positive: {len(falsy_map[cases[2]])} ({len(falsy_map[cases[2]])/len(truth_map[cases[2]])*100})')
    

    tp_negative = len(predictions_map[cases[0]]) - len(falsy_map[cases[0]])
    tp_neutral = len(predictions_map[cases[1]]) - len(falsy_map[cases[1]])
    tp_positive = len(predictions_map[cases[2]]) - len(falsy_map[cases[2]])
    
    fn_negative = len(truth_map[cases[0]]) - tp_negative
    fn_neutral = len(truth_map[cases[1]]) - tp_neutral
    fn_positive = len(truth_map[cases[2]]) - tp_positive
    
    
    f1_negative = calculate_f1(tp_negative, len(falsy_map[cases[0]]), fn_negative)
    f1_neutral = calculate_f1(tp_neutral, len(falsy_map[cases[1]]), fn_neutral)
    f1_positive = calculate_f1(tp_positive, len(falsy_map[cases[2]]), fn_positive)
    average_f1 = (f1_negative + f1_neutral + f1_positive) / 3

    print(f"F1 Score Negative: {f1_negative:.4f}")
    print(f"F1 Score Neutral: {f1_neutral:.4f}")
    print(f"F1 Score Positive: {f1_positive:.4f}")
    print(f"Average F1 Score: {average_f1:.4f}")
    print("============\n\n\n")

In [ ]:
def compare(model_result, model_preds, model_type, test_df):
    print(model_type)
    cases = ['negative', 'neutral', 'positive']

    predictions_map = {
        'negative': [],
        'neutral': [],
        'positive': [],
    }
    truth_map = {
        'negative': [],
        'neutral': [],
        'positive': [],
    }

    for i, (result, preds) in enumerate(zip(model_result, model_preds)):
        score = test_df['Sentiment'].iloc[i]
        truth_map[cases[score]].append(i)
        predictions_map[cases[preds]].append(i)

    total_data = len(test_df)

    print("Predictions")
    print(f'Negative: {len(predictions_map[cases[0]])} | Neutral: {len(predictions_map[cases[1]])} | Positive: {len(predictions_map[cases[2]])}')
    print("============\n")
    print("Truth")
    print(f'Negative: {len(truth_map[cases[0]])} | Neutral: {len(truth_map[cases[1]])} | Positive: {len(truth_map[cases[2]])}')
    print("============\n")

    tp_negative = len(set(predictions_map['negative']).intersection(set(truth_map['negative'])))
    fp_negative = len(predictions_map['negative']) - tp_negative
    fn_negative = len(truth_map['negative']) - tp_negative
    tn_negative = total_data - (tp_negative + fp_negative + fn_negative)

    tp_neutral = len(set(predictions_map['neutral']).intersection(set(truth_map['neutral'])))
    fp_neutral = len(predictions_map['neutral']) - tp_neutral
    fn_neutral = len(truth_map['neutral']) - tp_neutral
    tn_neutral = total_data - (tp_neutral + fp_neutral + fn_neutral)

    tp_positive = len(set(predictions_map['positive']).intersection(set(truth_map['positive'])))
    fp_positive = len(predictions_map['positive']) - tp_positive
    fn_positive = len(truth_map['positive']) - tp_positive
    tn_positive = total_data - (tp_positive + fp_positive + fn_positive)

    print("True Positives")
    print(f'Negative: {tp_negative} | Neutral: {tp_neutral} | Positive: {tp_positive}')
    print("True Negatives")
    print(f'Negative: {tn_negative} | Neutral: {tn_neutral} | Positive: {tn_positive}')
    print("False Negatives")
    print(f'Negative: {fn_negative} | Neutral: {fn_neutral} | Positive: {fn_positive}')
    print("False Positives")
    print(f'Negative: {fp_negative} | Neutral: {fp_neutral} | Positive: {fp_positive}')

    f1_negative = calculate_f1(tp_negative, fp_negative, fn_negative)
    f1_neutral = calculate_f1(tp_neutral, fp_neutral, fn_neutral)
    f1_positive = calculate_f1(tp_positive, fp_positive, fn_positive)
    average_f1 = (f1_negative + f1_neutral + f1_positive) / 3

    print(f"F1 Score Negative: {f1_negative:.4f}")
    print(f"F1 Score Neutral: {f1_neutral:.4f}")
    print(f"F1 Score Positive: {f1_positive:.4f}")
    print(f"Average F1 Score: {average_f1:.4f}")
    print("============\n\n\n")

In [ ]:
compare(bert_cnn_result.predictions, bert_cnn_preds,'bert-cnn')
compare(roberta_result.predictions, roberta_preds, roberta_model_type)
compare(trans_blstm_result.predictions, trans_blstm_preds, 'trans-blstm')

In [ ]:
# compare(bert_cnn_result.predictions, bert_cnn_preds,'bert-cnn')